In [1]:
# pip install yt-dlp openai-whisper hugchat

In [1]:
## Importing Libraries

import os
import openai
import pandas as pd
import numpy as np
import tiktoken
import time
from tqdm import tqdm
# from langchain.agents import create_csv_agent
# from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain.document_loaders import UnstructuredPowerPointLoader

import yt_dlp
import whisper
from hugchat import hugchat
from whisper.utils import get_writer
import os

# set your openai key here
key = "5d69dd6f0aa14121ae81066435443abf"
AZURE_OPENAI_SERVICE = "OpenAI-PoC-East"
openai_api_version = "2023-03-15-preview" 
openai_api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com" 
openai_api_key = key
openai.api_type = "azure"
openai.api_version = openai_api_version 
openai.api_base = openai_api_base # Your Azure OpenAI resource's endpoint value.
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key  

In [2]:
## API Function
def api_call(input_str, temp, prompt):
    temp = temp
    prompt = prompt
    response = openai.ChatCompletion.create(
        engine="gptdemo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": input_str}
        ], 
        temperature = temp
    )
    return (response)

In [3]:
# #Function for saving audio from input video id of youtube
def download(video_id: str) -> str:
    # video_url = f'https://www.youtube.com/watch?v={video_id}'
    video_url = video_id
    ydl_opts = {
        'ffmpeg_location':os.path.realpath('C:\\ffmpeg\\ffmpeg-2024-03-04-git-e30369bc1c-full_build\\bin\\ffmpeg.exe'),
        'format': 'wav/bestaudio/best',
        'paths': {'home': 'audio/'},
        'outtmpl': {'default': '%(id)s.%(ext)s'},
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([video_url])
        if error_code != 0:
            raise Exception('Failed to download video')

    return f'audio/{video_id}.wav'


#Call function with video id
# video_id = 'CtsRRUddV2s&t=19s&ab_channel=VisuallyExplained'
# video_id = 'https://www.youtube.com/watch?v=CtsRRUddV2s&t=25s&ab_channel=VisuallyExplained'
# video_id = 'https://www.youtube.com/watch?v=qxo8p8PtFeA&ab_channel=IBMTechnology'
# file_path = download(video_id)

# file_path = download('A_JQK_k4Kyc&t=99s')

In [4]:
files = os.listdir('audio')

In [5]:
# Load whisper model
whisper_model = whisper.load_model("tiny")

# Transcribe audio function
def transcribe(file_path: str) -> str:
  # `fp16` defaults to `True`, which tells the model to attempt to run on GPU.
  transcription = whisper_model.transcribe(file_path, fp16=False)
  return transcription['text']
  

#Call the transcriber function with file path of audio  
# transcript = transcribe('C:\\Users\\akurapa\\OneDrive - Daiichi Sankyo\\Documents\\Notebooks\\video_analyzer\\audio\\CtsRRUddV2s.m4a')
# transcript = transcribe('C:/Users/akurapa/OneDrive - Daiichi Sankyo/Documents/Notebooks/streamlit_video_analyzer/audio/CtsRRUddV2s.m4a')
# transcribe('audio\qxo8p8PtFeA.m4a')
# transcript = transcribe('C:/Users/akurapa/OneDrive - Daiichi Sankyo/Documents/Notebooks/csv_upload/audio/CtsRRUddV2s.mp3')
# print(transcript)

# # Save as a TXT file without any line breaks
# with open("transcription_a.txt", "w", encoding="utf-8") as txt:
#     txt.write(transcript)

In [6]:
# video_id = 'https://www.youtube.com/watch?v=qxo8p8PtFeA&ab_channel=IBMTechnology'
video_id = 'https://www.youtube.com/watch?v=OKB1PuclFEo&t=1s&ab_channel=Zuby'
download(video_id)
# files = os.listdir('audio')
# for file_path in files:
    # print(file_path)
# transcribe('audio/qxo8p8PtFeA.m4a')
    # print(transcribe)

[youtube] Extracting URL: https://www.youtube.com/watch?v=OKB1PuclFEo&t=1s&ab_channel=Zuby
[youtube] OKB1PuclFEo: Downloading webpage
[youtube] OKB1PuclFEo: Downloading ios player API JSON
[youtube] OKB1PuclFEo: Downloading android player API JSON


[youtube] OKB1PuclFEo: Downloading android player API JSON


[youtube] OKB1PuclFEo: Downloading android player API JSON


[youtube] OKB1PuclFEo: Downloading android player API JSON


[youtube] OKB1PuclFEo: Downloading player 589f1394
[youtube] OKB1PuclFEo: Downloading m3u8 information
[info] OKB1PuclFEo: Downloading 1 format(s): 251
[download] Destination: audio\OKB1PuclFEo.webm
[download] 100% of   14.20MiB in 00:00:00 at 21.88MiB/s    
[ExtractAudio] Destination: audio\OKB1PuclFEo.wav
Deleting original file audio\OKB1PuclFEo.webm (pass -k to keep)


'audio/https://www.youtube.com/watch?v=OKB1PuclFEo&t=1s&ab_channel=Zuby.wav'

In [11]:
with open("transcription.txt", "r", encoding="utf-8") as txt:
    content = txt.read()


 An important task in machine learning is prediction. Given some information summarizing an independent variable x, for example the height of a person, predictive value of another dependent variable y, like their weights. Usually we have a training dataset, that is a table that contains values for both x and y, that we want to use to infer what the function g should be. We can then use that learned function g to predict the values y for new values of x, not seen in training. Find the right function g is called regression, and easiest way to do this is to assume that this function g is a linear function, hence the name linear regression. Here alpha is the slope of this line, and beta is the intrusive. The simplicity of linear regression makes it really attractive. We're only dealing with linear functions here, so no complicated nor network's involved. If you are faced with a prediction problem, linear regression should really be the first thing that you try. To do linear regression, we 

In [13]:
system_prompt = """You will provide information for the user input based on the below content
                {0}""".format(content)

In [20]:
# user_prompt = "Summarize the information"
user_prompt = "Provide key points in bullets"
temp = 0.5

In [21]:
response = api_call(user_prompt, temp, system_prompt)
print(response['choices'][0]['message']['content'])

- Prediction is an important task in machine learning
- Linear regression is a method for finding a linear function that predicts the value of a dependent variable based on an independent variable
- Linear regression assumes that the function g is a linear function
- Alpha is the slope of the line, and beta is the intercept
- The goal of linear regression is to find the values of alpha and beta that make the line fit the data as closely as possible
- The fit of the line to the data can be quantified using the sum of the squared differences between the actual values and the predicted values
- Linear regression is simple and tractable, and can be extended to handle more than one input variable or non-linear relationships
- The coefficients of the linear regression model can be interpreted to understand the relationship between the input and output variables
- Linear regression is a good first method to try for prediction problems in machine learning.


In [21]:
pip install moviepy

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     ------------------------------------- 388.3/388.3 kB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/313.4 kB ? eta -:--:--
     ------------------------------------- 313.4/313.4 kB 20.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
     --------- ----------------------------- 5.5/22.6 MB 116.1 MB/s eta 0:00:01
     ------------------ ------------------- 11.1/22.6 MB 131.2 MB/s eta 0:00:01
     --------------------- ----------------- 12.6/22.6 MB 81.8 MB/s eta 0:00:01
     --------------------------------- ----- 19.4/22.6 MB 81.8 MB/s eta 0:00:01
     -------------------------------------  22.6/22.6 MB 108.8 MB/s eta 0:00:01
     -------------------------------------  22.6/22.6 MB 108.8 MB/s eta 0:00:01
     -----------------------------------


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\akurapa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
